In [2]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796908 sha256=9fd71f90ecaa9af640d3e52a6395f0e80a3b5ae11470f491144a7a2ba7cd3b24
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper


**There are 5 model sizes, offering speed and accuracy tradeoffs.**
1. **Tiny** - 39M parameters, 1GB VRAM Required, 32x Speed
2. **Base** - 74M parameters, 1GB VRAM Required, 16x Speed
3. **Small** - 244M parameters, 2GB VRAM Required, 6x Speed
4. **Medium** - 769M parameters, 5GB VRAM Required, 2x Speed
5. **Large** - 1550M parameters, 10GB VRAM Required, 1x Speed

In [3]:
import whisper
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 59.3MiB/s]


In [4]:
result = model.transcribe("harvard.wav", verbose=True)
result['text']

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:04.480]  The stale smell of old beer lingers.
[00:04.480 --> 00:07.020]  It takes heat to bring out the odor.
[00:07.020 --> 00:09.940]  A cold dip restores health and zest.
[00:09.940 --> 00:12.620]  A salt pickle tastes fine with ham.
[00:12.620 --> 00:15.080]  Tacos al pastor are my favorite.
[00:15.080 --> 00:17.600]  A zestful food is the hot cross bun.


' The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.'

In [5]:
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 4.48,
  'text': ' The stale smell of old beer lingers.',
  'tokens': [50364,
   440,
   342,
   1220,
   4316,
   295,
   1331,
   8795,
   22949,
   433,
   13,
   50588],
  'temperature': 0.0,
  'avg_logprob': -0.25378372420125933,
  'compression_ratio': 1.4210526315789473,
  'no_speech_prob': 0.022392641752958298},
 {'id': 1,
  'seek': 0,
  'start': 4.48,
  'end': 7.0200000000000005,
  'text': ' It takes heat to bring out the odor.',
  'tokens': [50588, 467, 2516, 3738, 281, 1565, 484, 264, 41176, 13, 50715],
  'temperature': 0.0,
  'avg_logprob': -0.25378372420125933,
  'compression_ratio': 1.4210526315789473,
  'no_speech_prob': 0.022392641752958298},
 {'id': 2,
  'seek': 0,
  'start': 7.0200000000000005,
  'end': 9.94,
  'text': ' A cold dip restores health and zest.',
  'tokens': [50715, 316, 3554, 10460, 1472, 2706, 1585, 293, 37889, 13, 50861],
  'temperature': 0.0,
  'avg_logprob': -0.25378372420125933,
  'compression_ratio': 1

In [7]:
for i, seg in enumerate(result['segments']):
  print(i+1, " -", seg['text'])

1  -  The stale smell of old beer lingers.
2  -  It takes heat to bring out the odor.
3  -  A cold dip restores health and zest.
4  -  A salt pickle tastes fine with ham.
5  -  Tacos al pastor are my favorite.
6  -  A zestful food is the hot cross bun.


In [8]:
import pandas as pd
speech = pd.DataFrame.from_dict(result['segments'])
speech.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,4.48,The stale smell of old beer lingers.,"[50364, 440, 342, 1220, 4316, 295, 1331, 8795,...",0.0,-0.253784,1.421053,0.022393
1,1,0,4.48,7.02,It takes heat to bring out the odor.,"[50588, 467, 2516, 3738, 281, 1565, 484, 264, ...",0.0,-0.253784,1.421053,0.022393
2,2,0,7.02,9.94,A cold dip restores health and zest.,"[50715, 316, 3554, 10460, 1472, 2706, 1585, 29...",0.0,-0.253784,1.421053,0.022393
3,3,0,9.94,12.62,A salt pickle tastes fine with ham.,"[50861, 316, 5139, 31433, 8666, 2489, 365, 785...",0.0,-0.253784,1.421053,0.022393
4,4,0,12.62,15.08,Tacos al pastor are my favorite.,"[50995, 38848, 329, 419, 21193, 366, 452, 2954...",0.0,-0.253784,1.421053,0.022393


In [9]:
audio = whisper.load_audio('harvard.wav')
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
_, probs = model.detect_language(mel)

In [10]:
sorted(probs.items(), key=lambda x:x[1], reverse=True)[:10]

[('en', 0.9977268576622009),
 ('ja', 0.0006539112655445933),
 ('nn', 0.0003194053133483976),
 ('ko', 0.00021457891853060573),
 ('pt', 0.0001776807475835085),
 ('es', 0.00011555601668078452),
 ('la', 0.00010099864448420703),
 ('fr', 8.328038529725745e-05),
 ('haw', 7.064113742671907e-05),
 ('de', 6.923016917426139e-05)]

**Working on an Earnings Call Data File for Transcription**

In [1]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 858.9 kB/s eta 0:00:00


In [4]:
from pytube import YouTube

In [6]:
youtube_video_url = "https://www.youtube.com/watch?v=3haowENzdLo"
youtube_video_content = YouTube(youtube_video_url)

In [7]:
audio_streams = youtube_video_content.streams.filter(only_audio=True)
for stream in audio_streams:
  print(stream)

<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">
<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
<Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">


In [8]:
audio_stream = audio_streams[1]
print(audio_stream)

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">


In [9]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [12]:
audio_stream.download("earnings_call_microsoft_q4_2022.mp4")

'/content/earnings_call_microsoft_q4_2022.mp4/Microsoft (MSFT) Q4 2022 Earnings Call.mp4'

In [13]:
!ffmpeg -ss 1924 -i "/content/earnings_call_microsoft_q4_2022.mp4/Microsoft (MSFT) Q4 2022 Earnings Call.mp4" -t 2515 "earnings_call_microsoft_q4_2022_filtered.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [14]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/content/earnings_call_microsoft_q4_2022_filtered.mp4")
result["text"]

" Good afternoon and thank you for joining us today. On the call with me are Satin Adela, Chairman and Chief Executive Officer, Amy Hood, Chief Financial Officer, Alice Chala, Chief Accounting Officer, and Keith Tholiver, Deputy General Counsel. On the Microsoft Invest Relations website, you can find our earnings press release and financial summary slide deck, which is intended to supplement our prepared remarks during today's call, and provides the reconciliation of differences between gap and non-gap financial measures. Unless otherwise specified, we will refer to non-gap metrics on the call. The non-gap financial measures provided should not be considered as a sub-seat four, or superior to the measures of financial performance prepared in accordance with gap. They are included as additional clarifying items to aid investors and further understanding the company's fourth quarter performance in addition to the impact these items and events have on the financial results. All growth com